# Zajęcia 1.

Zakres:
* zapoznanie sie z 2 srodowiskami pracy (klaster Hadoop oraz środowisko dostepne na GCP.
* operacje na rozproszonym i obiektowym systemie plików (HDFS i GCS)
* wykorzystanie platformy Apache Spark

Języki:
* bash, python, sql






Krótka lista przydatnych poleceń bash:

https://www.reddit.com/r/linux/comments/9rns12/some_linux_commands_cheatsheet/


URL Jupyter na klastrze Instytutu Informatyki: https://zsibio.ii.pw.edu.pl/jupyter/


## Klaster Hadoop

edge node: cdh00 (węzeł dostępowy)

HDFS:
name node: cdh01
data nodes: cdh02-cdh05

YARN
resource manager: cdh01
node managers: cdh02-cdh05

konsola administracyjna: http://cdh01:8080/

Dystrybucja Hadoop: Hortonworks




## Jupyter Notebook

Interaktywny notatnik dostepny poprzez przeglądarkę.
Służy do wpisywania poleceń w wybranych jezykach programowania oraz opis w tzw jezyku markdown.
Tryb edycji + tryb poleceń. 
Podstawowe skróty klawiaturowe: 
* Esc/Enter
* strzalki
* ctrl-enter/shift-enter
* a/b/d
* y/m 
* znaczenie ! oraz %%
Notatnik zapisywany jest w formacie ipynb (IPythonNoteBook)

Kolejnosc uruchomienia moze byc rozna.
Mozliwosc zatrzymania kernela i uruchomienia od nowa. 

Polecamy film: https://www.youtube.com/watch?v=HW29067qVWk



### Security
Klaster jest zabezpieczony przed niepowolanym dostepem.
Zazwyczaj dostep do klastra to: zalogowanie poprzez ssh do wezla dostepowego a potem wykonywanie polecen. 
My logujemy sie do Jupytera, w tle pobierany jest ticket dostepowy do odpowiednich zasobów klastra. Korzystajac ze zmiennych srodowiskowych i plikow konfiguracyjnych Jupyter wie jak polaczyc sie z menedzerem zasobow oraz namenodem HDFS.

Dostep do Jupyter jest publiczny i po zajeciach mozna z niego korzystac.


## Praca z rozproszonym systemem plików

In [ ]:
import os
user_name = os.environ.get('USER')
print(user_name) 

## Kopiowanie danych do swojego HOME

W katalogu `/data/local/datascience/data/` znajduje sie plik który bedzie nam dzisiaj słuzyl do pracy. Należy go skopiować do swojego katalogu domowego. 


In [ ]:
%%bash
cd 
mkdir -p data
cd data
cp /data/local/datascience/data/brca.txt .
ls


In [ ]:
%%bash
head data/brca.txt
echo
tail data/brca.txt
echo
wc -l data/brca.txt

## HDFS
Do tej pory korzystalismy z lokalnego systemu plików do ktorego ma dostep maszyna na ktora sie zalogowalismy (cdh00). Teraz zaczniemy korzystac z rozproszonego systemu plikow.

Dla podstawowych polecen systemowych na plikach istnieja odpowiedniki polecen dla systemu HDFS
* `ls` -> `hdfs dfs -ls`
* `cp` -> `hdfs dfs -cp`
* `mv` -> `hdfs dfs -mv`
* `rm` -> `hdfs dfs -rm`

Pełna lista poleceń znajduje się na www: https://hadoop.apache.org/docs/r2.4.1/hadoop-project-dist/hadoop-common/FileSystemShell.html

### Listowanie swojego HOME na HDFS
Każdy ma swoj katalog domowy na HDFS


In [ ]:
! hdfs dfs -ls 

Swoj katalog mozna tez sprawdzic podajac sciezke bezwgledna od katalogu glownego

In [ ]:
! hdfs dfs -ls /user/${USER}

Sprawdzmy katalog glowny na HDFS

In [ ]:
!hdfs dfs -ls 

### ZADANIE 1
Korzystajac z analogii do polecen znanych z lokalnego sytemu pliku oraz z dokumentacji polecen dla HDFS stworz na HDFS katalog external a w nim katalog data w swoim podkatalogu domowym. Sprawdz foldery istnieją.

In [ ]:
%%bash
hdfs dfs -mkdir -p /user/${USER}/external/data 2>/dev/null
hdfs dfs -ls -R /user/${USER}/external/ 2>/dev/null

### Dodanie pliku na HDFS

In [ ]:
%%bash
hdfs dfs -put data/brca.txt /user/${USER}/external/data
hdfs dfs -ls /user/${USER}/external/data

Jaki jest rozmiar pliku? Mozna skorzystac z przelacznika -h w ls

In [ ]:
!hdfs dfs -ls -h /user/${USER}/external/data

A calkowita wielkosc pliku? Polecenie `du`

In [ ]:
!hdfs dfs -du -h /user/${USER}/external/data/brca.txt

Informacje o statusie pliku: polecenie `fsck`

In [ ]:
!hdfs fsck /user/${USER}/external/data/brca.txt

Plik mozna rowniez odczytac bezposrednio z HDFS:

In [ ]:
! hdfs dfs -cat /user/${USER}/external/data/brca.txt | head 

### ZADANIE 2
W swoim katalogu domowym stworz katalog external/temp. Skopiuj do niego plik brca.txt 

In [ ]:
%%bash
hdfs dfs -mkdir -p /user/${USER}/external/temp
hdfs dfs -cp /user/${USER}/external/data/brca.txt /user/${USER}/external/temp
hdfs dfs -ls /user/${USER}/external/temp

### Usuwanie i odzyskiwanie plikow

Do usuwanie sluzy polecenie -rm. Domyslnie usuwany plik jest przesuwany do kosza, skad mozna go odzyskac. 


In [ ]:
%%bash
hdfs dfs -rm /user/${USER}/external/temp/brca.txt
hdfs dfs -ls /user/${USER}/external/temp/

Dostalismy lokalizacje pliku z kosza, stamtad mozemy go skopiowac.

Do samodzielnej weryfikacji:
* Przywrócenie pliku z kosza
* Zachowanie polecenia rm przy podaniu parametru `skipTrash`.

### Zmiana uprawnień na katalogu/pliku


In [ ]:
%%bash
hdfs dfs -chmod -R 747 /user/${USER}/external/data/
hdfs dfs -ls /user/${USER}/external/data/


### Pobranie pliku z powrotem na lokalny dysk

In [ ]:
!hdfs dfs -get /user/${USER}/external/data/brca.txt brca2.txt

In [ ]:
!ls

## Apache Spark
podłączenie się do klastra Sparkowego
stworzenie tabeli z pliku CSV
zapytania SQL na tabelach (SparkSQL)
Na tych cwiczeniach bedziemy korzystac z pliku ktory zapisalismy na HDFS w pierwszej czesci cwicczenia.

Sprawdzmy czy plik znajduje się na swoim miejscu w katalogu: /user/${USER}/external/data

TODO - dalabym przyklad recznego stworzenia DF. Potem stworzenie DF na podstawie danych (?)

In [ ]:
! hdfs dfs -ls /user/${USER}/external/data

Losujemy numer portu. Domyslnie port na ktorym Spark wystawia swoj interfejs graficzny jest 4040. zeby nie zajmowac sobie wzajmnie numerow portu - wykonamy losowanie.


In [ ]:
import os
user_name = os.environ.get('USER')
print(user_name)

In [ ]:
import random
ui_port = random.randint(4000,4999)
print(ui_port)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.master('yarn-client') \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.config('spark.ui.port',f'{ui_port}') \
.appName(f'ds-{user_name}') \
.getOrCreate()

Wczytanie danych
Będziemy wykonywać analizę na danych z https://gdac.broadinstitute.org/ zawierajacych dane z badani nad CNV w obrebie BRAC2. Ściągniety plik znajduje sie w hdfs://edugen/brca.txt TODO

    

In [ ]:
path = f'/user/{user_name}/external/data/brca.txt'


In [ ]:
df = spark.read.load(path, format="csv", sep="\t", inferSchema="true", header="true")

# df.registerTempTable(f"{table_name}")

In [ ]:
type (df)

In [ ]:
df.printSchema()


In [ ]:
df.show()
df.count() # wymiary
len(df.columns) # wymiary
df.head (5) # check
df.describe().show()

In [ ]:
df.select("Sample").show()


In [ ]:
df_chrom = df.select("Chromosome").distinct()
df_chrom.count()
df.filter(df.Chromosome > 21).show()

In [ ]:
df_cached.groupBy("Chromosome").count().show()

In [ ]:
df_cached.groupBy("Sample","Chromosome").count().orderBy(asc("Sample")).show()

In [ ]:
# add withColumn

In [ ]:
gs://bdg-la

### Przeniesienie pliku na GCS

In [ ]:
! gsutil cp ~/ds-notebooks/README.md gs://bdg-lab-$USER  # upload obiektu do kubełka